In [1]:
library(reticulate)
library(glmnet)
library(ppcor)
library(kSamples)

Loading required package: Matrix

Loaded glmnet 4.1-2

Loading required package: MASS

Loading required package: SuppDists



In [2]:
os <- import("os")

## SYNTHETIC DATA

In [3]:
times_synthetic <- c()

In [3]:
names <- c('dyn-BF', 'dyn-BFC', 'dyn-CY', 'dyn-LI', 'dyn-LL', 'dyn-TF')

In [6]:
names_synthetic <- c()
for (name in names){
    base_path <- gsub(" ", "", paste('./beeline_data/synthetic/',name,'/'), fixed=TRUE)
    for (folder in os$listdir(base_path)){
        names_synthetic <- c(names_synthetic, folder)
    }}

In [7]:
write(names_synthetic, "names_synthetic.txt")

In [5]:
for (name in names){
    
    #start time
    start_time <- Sys.time()
    
    base_path <- gsub(" ", "", paste('./beeline_data/synthetic/',name,'/'), fixed=TRUE)
    
    for (folder in os$listdir(base_path)) {
        temp_path <- os$path$join(base_path, folder)
        print(temp_path)
        if (os$path$isdir(temp_path)){
            
            #read expression data
            expression_path <- os$path$join(temp_path,'ExpressionData.csv')
            gem <- read.csv(expression_path)
            gem <- t(gem)
            colnames(gem) <- gem[1,]
            gem <- gem[-1,]
            
            #read time data
            time_path <- os$path$join(temp_path,'PseudoTime.csv')
            pseudotime <- read.csv(time_path)
            rownames_temp <- pseudotime[,1]
            pseudotime <- pseudotime[,-1]
            pseudotime <- as.data.frame(pseudotime)
            rownames(pseudotime) <- rownames_temp
            
            #format pseudotime dataframe
            pseudotime[is.na(pseudotime)] <- 0
            pseudotime <- rowSums(pseudotime, na.rm = FALSE, dims = 1)
            pseudotime <- as.data.frame(pseudotime)
            
            #add time data to the expression dataframe and sort it according
            gem <- cbind(gem,pseudotime)
            gem <- gem[order(gem$pseudotime),]
            gem <- as.data.frame(sapply(gem,as.numeric))
            
            #SINCERITIES Data Preprocessing
            totDATA <- gem[,1:dim(gem)[2]-1]
            timeline <- gem[,dim(gem)[2]]
            DATA.time <- sort(unique(timeline))
            DATA.num_time_points <- length(DATA.time)
            DATA.totDATA <- matrix(ncol = dim(gem)[2]-1)
            DATA.timeline <- vector()
            
            for (k in 1:DATA.num_time_points) {
                I <- which(timeline==DATA.time[k])
                DATA.totDATA <- rbind(DATA.totDATA,as.matrix(totDATA[I,]))
                DATA.timeline <- c(DATA.timeline,timeline[I])
            }
            
            DATA.totDATA <- DATA.totDATA[-1,]
            DATA.totDATA[is.na(DATA.totDATA)] <- 0
            DATA.numGENES <- dim(DATA.totDATA)[2]
            DATA.genes <- colnames(gem)[1:dim(gem)[2]-1]
            DATA.singleCELLdata <- by(DATA.totDATA,DATA.timeline,identity)
            DATA <- list(time=DATA.time,num_time_points=DATA.num_time_points,
                         totDATA=DATA.totDATA,timeline=DATA.timeline,numGENES=DATA.numGENES,
                         genes=DATA.genes,singleCELLdata=DATA.singleCELLdata)
            
            #Run SINCERITIES
            SINCERITITES <- dget("SINCERITIES.R")
            result <- SINCERITITES(DATA,distance=1,method = 1,noDIAG = 0,SIGN = 0)
            
            adj_matrix <- result$adj_matrix/max(result$adj_matrix)
            final_ranked_predictions <- dget("final_ranked_predictions.R")
            table <- final_ranked_predictions(adj_matrix,DATA$genes,fileNAME=sprintf(folder),saveFile = TRUE)
            
            #end time
            end_time <- Sys.time()
            
            times_synthetic <- c(times_synthetic, end_time - start_time)
        }
    }
}

[1] "./beeline_data/synthetic/dyn-BF/dyn-BF-2000-4"


Warning message in dir.create("Results"):
“'Results' already exists”


[1] "./beeline_data/synthetic/dyn-BF/dyn-BF-5000-5"


Warning message in ks.test(p1, p2):
“cannot compute exact p-value with ties”
Warning message in ks.test(p1, p2):
“cannot compute exact p-value with ties”
Warning message in dir.create("Results"):
“'Results' already exists”


[1] "./beeline_data/synthetic/dyn-BF/dyn-BF-2000-6"


...

Warning message in ks.test(p1, p2):
“cannot compute exact p-value with ties”
Warning message in dir.create("Results"):
“'Results' already exists”


[1] "./beeline_data/synthetic/dyn-TF/dyn-TF-2000-7"


Warning message in dir.create("Results"):
“'Results' already exists”


[1] "./beeline_data/synthetic/dyn-TF/dyn-TF-2000-9"


Warning message in dir.create("Results"):
“'Results' already exists”


In [6]:
times_synthetic

ERROR: Error in eval(expr, envir, enclos): object 'times_synthetic' not found


## CURATED DATA

In [ ]:
times_curated <- c()

In [8]:
names <- c('GSD', 'HSC', 'mCAD', 'VSC')

In [9]:
names_curated <- c()
for (name in names){
    base_path <- gsub(" ", "", paste('./beeline_data/curated/',name,'/'), fixed=TRUE)
    for (folder in os$listdir(base_path)){
        names_curated <- c(names_curated, folder)
    }}

In [10]:
write(names_curated, "names_curated.txt")

In [ ]:
for (name in names){
    
    #start time
    start_time <- Sys.time()
    
    #bases path
    base_path <- gsub(" ", "", paste('./beeline_data/curated/',name,'/'), fixed=TRUE)
    
    #iterate through all the folders
    for (folder in os$listdir(base_path)) {
        temp_path <- os$path$join(base_path, folder)
        print(temp_path)
        if (os$path$isdir(temp_path)){
            
            #read expression data
            expression_path <- os$path$join(temp_path,'ExpressionData.csv')
            gem <- read.csv(expression_path)
            gem <- t(gem)
            colnames(gem) <- gem[1,]
            gem <- gem[-1,]
            
            #read time data
            time_path <- os$path$join(temp_path,'PseudoTime.csv')
            pseudotime <- read.csv(time_path)
            rownames_temp <- pseudotime[,1]
            pseudotime <- pseudotime[,-1]
            pseudotime <- as.data.frame(pseudotime)
            rownames(pseudotime) <- rownames_temp
            
            #format pseudotime dataframe
            pseudotime[is.na(pseudotime)] <- 0
            pseudotime <- rowSums(pseudotime, na.rm = FALSE, dims = 1)
            pseudotime <- as.data.frame(pseudotime)
            
            #add time data to the expression dataframe and sort it according
            gem <- cbind(gem,pseudotime)
            gem <- gem[order(gem$pseudotime),]
            gem <- as.data.frame(sapply(gem,as.numeric))
            
            #SINCERITIES Data Preprocessing
            totDATA <- gem[,1:dim(gem)[2]-1]
            timeline <- gem[,dim(gem)[2]]
            DATA.time <- sort(unique(timeline))
            DATA.num_time_points <- length(DATA.time)
            DATA.totDATA <- matrix(ncol = dim(gem)[2]-1)
            DATA.timeline <- vector()
            
            for (k in 1:DATA.num_time_points) {
                I <- which(timeline==DATA.time[k])
                DATA.totDATA <- rbind(DATA.totDATA,as.matrix(totDATA[I,]))
                DATA.timeline <- c(DATA.timeline,timeline[I])
            }
            
            DATA.totDATA <- DATA.totDATA[-1,]
            DATA.totDATA[is.na(DATA.totDATA)] <- 0
            DATA.numGENES <- dim(DATA.totDATA)[2]
            DATA.genes <- colnames(gem)[1:dim(gem)[2]-1]
            DATA.singleCELLdata <- by(DATA.totDATA,DATA.timeline,identity)
            DATA <- list(time=DATA.time,num_time_points=DATA.num_time_points,
                         totDATA=DATA.totDATA,timeline=DATA.timeline,numGENES=DATA.numGENES,
                         genes=DATA.genes,singleCELLdata=DATA.singleCELLdata)
            
            #Run SINCERITIES
            SINCERITITES <- dget("SINCERITIES.R")
            result <- SINCERITITES(DATA,distance=1,method = 1,noDIAG = 0,SIGN = 0)
            
            adj_matrix <- result$adj_matrix/max(result$adj_matrix)
            final_ranked_predictions <- dget("final_ranked_predictions.R")
            table <- final_ranked_predictions(adj_matrix,DATA$genes,fileNAME=sprintf(folder),saveFile = TRUE)
            
            #end time
            end_time <- Sys.time()
            
            times_curated <- c(times_curated, end_time - start_time)

        }
    }
}